In [1]:
import pandas as pd

data = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
real = pd.read_csv("./sample_submission.csv")

In [2]:
# NE DIRAJ
# Preturio sam train.csv da vidim koje String vrednosti mogu da se prebace u Int, ostatak nema smisla da se konvertuje

toReplace = {'GLQ' : 6,  
             'ALQ' : 5, 'Ex' : 5,
             'BLQ' : 4, 'Gd' : 4, 'GdPrv' : 4,
             'Rec' : 3, 'TA' : 3, 'Av' : 3, 'Fin' : 3, 'MnPrv' : 3,
             'LwQ' : 2, 'Fa' : 2, 'Mn' : 2, 'RFn' : 2, 'GdWo' : 2,
             'Unf' : 1, 'Po' : 1, 'No' : 1, 'Unf' : 1, 'MnWw' : 1,
             'NA' : 0
            }

In [3]:
y = data.SalePrice
data = data.drop('SalePrice', axis=1)

for feature in data.columns: # Prolazim kroz svaku kolonu da bih konvertovao 
    data[feature] = data[feature].replace(toReplace) # String -> Int, ne moze racun da se radi nad stringom, a moze biti od koristi
    
#data = data.drop('Id', axis=1) # totalno nebitno
data = data.drop('Fence', axis=1) # preko 80% je NA
data = data.drop('PoolQC', axis=1) # preko 80% je NA
data = data.drop('FireplaceQu', axis=1) # preko 80% je NA

dataCategoricalFeatures = data.select_dtypes(include=['object']).columns.tolist() # odvojio nenumericke vrednosti i numericke
dataNumericalFeatures = data.select_dtypes(include=['int64', 'float64']).columns.tolist() 

# jedna kolona je kriticna, oko 20% podataka fali, ostalim fali 3% podataka
# sva prazna polja su popunjena medijanom
for feature in dataNumericalFeatures:
    data[feature].fillna(data[feature].mean(), inplace=True)

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

In [5]:
# testiranje

X = data[dataNumericalFeatures]
model = DecisionTreeRegressor()
model.fit(X, y)

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed

DecisionTreeRegressor()

In [6]:
prediction = pd.DataFrame({'Id': test['Id']})
#test = test.drop('Id', axis=1) # totalno nebitno
test = test.drop('Fence', axis=1) # zato sto nemamo tu kolonu
test = test.drop('PoolQC', axis=1) # zato sto nemamo tu kolonu
test = test.drop('FireplaceQu', axis=1) # zato sto nemamo tu kolonu

for feature in test.columns: 
    test[feature] = test[feature].replace(toReplace)

testCategoricalFeatures = test.select_dtypes(include=['object']).columns.tolist() # odvojio nenumericke vrednosti i numericke
testNumericalFeatures = test.select_dtypes(include=['int64', 'float64']).columns.tolist() 

for feature in testNumericalFeatures:
    test[feature].fillna(test[feature].mean(), inplace=True)

In [7]:

X = test[testNumericalFeatures]
#X_csv = pd.DataFrame(X)
#X.to_csv("./X.csv")

SalePrice = model.predict(X)
prediction.add({'SalePrice': SalePrice})
prediction.to_csv("./submission.csv")

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [8]:

X = test[testNumericalFeatures]

SalePrice = model.predict(X)
prediction['SalePrice'] = SalePrice
prediction.to_csv("./submission.csv")

/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/balkanskihaker/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [10]:
absoluteError = pd.DataFrame({'Id': prediction['Id'], 'Predicted': prediction['SalePrice'], 'Real': real['SalePrice'], 'Absolute error': abs(prediction['SalePrice']-real['SalePrice']) })
#absoluteError.to_csv('./absoluteError.csv')
#absoluteError['Absolute error'].mean() 
#medijana je oko 55.000 sto je velika greska, ali opet bolje nego nista

54957.1062381807